In [9]:
import numpy as np
import pandas as pd
#equations
n = int(input("Enter the number of equations : "))
#input target fucntion
z = [float(x) for x in input("coefficients of objective function: ").split()] 
z = np.array(z)
A = []
#matrix elements entering
print("Enter the coefficient matrix with space: ")

for i in range(0,n):
    row = [float(x) for x in input().split()]
    #add the row to A
    A.append(row)
A = np.array(A)
#add resource vector (dimension is not determined yet!)
b =  [float(x) for x in input("Enter the resources: ").split()]
b = np.array(b)
#column==>axis=1
b = b.reshape([n, 1])
#converte matrix into all positive matrix without any minus element
if(A.min()<0):
    A = A - A.min() + 1
print("The matrix with positive values is: ")
print(A)
#identity matrix , diagonal=1 , other elements=0
S = np.identity(n)
#merge 3 matrixes and creat 'matrix'
matrix = np.concatenate((A, S, b), axis=1)
m = A.shape[1]
#because the coefficients of z are not equal so we should find variables
# we work on column so column == count of variables

z = z.reshape([m, 1])

#we need -Z in the table
# count of zero== n+1
#count of b==1
Z = np.append(-z, np.zeros((1, n+1)))
Z = Z.reshape([1, n + m +1])
#consider rows as horizontal display
matrix = np.concatenate((matrix, Z), axis=0)
#converting to datafram for better displaying
matrix = pd.DataFrame(matrix)
#lable inserting in the table 
#add two strings
for i in range(0,n):
    matrix = matrix.rename(columns={i+m: 'S'+str(i+1)}, index={i: 'S'+str(i+1)})
    #X's lable
for i in range(0,m):
    matrix = matrix.rename(columns={i: 'X'+str(i+1)})
    #the last line lable:Z
matrix = matrix.rename(index={n: 'Z'})
#the last column
matrix = matrix.rename(columns={m+n: 'b'})
display(matrix)

#finding the pivot column by finding the location of minimum Z (column of minimum amount of Z)
pivotCol = matrix.loc['Z'].idxmin(axis=0, skipna=True)
finish = False
#divide a distinct column("bb") into column b in a loop 
while(matrix[pivotCol].loc['Z'] < 0 and not(finish)):
    #store only positive amounts and through out the minus and zero elements
    bb = matrix['b']/matrix[pivotCol]
    bb = bb[bb > 0] 
    #if bb is empty so the algorithm is finished
    if(bb.empty):
        finish = True
        #else.. determine the pivot row 
    else:
        pivotRow = bb.idxmin(axis=1, skipna=True)
        #Now we have PIVOT :)
        pivot = matrix[pivotCol].loc[pivotRow]
        #function for coloring a cell in table (green)
        def style_specific_cell(x):
            color = 'background-color: lightgreen'
            df1 = pd.DataFrame('', index=x.index, columns=x.columns)
            df1[pivotCol].loc[pivotRow] = color
            return df1
        display(matrix.style.apply(style_specific_cell, axis=None))
         #pivot must be 1 so pivot/pivot
        matrix.loc[pivotRow] = matrix.loc[pivotRow] / pivot
        #pivot column except pivot becomes zero by multipling and adding rows etc.

        rowNames = matrix.index.values[matrix.index.values != pivotRow]
        for index in rowNames:
            coef = -matrix[pivotCol].loc[index]
            matrix.loc[index] = matrix.loc[pivotRow] * coef + matrix.loc[index]
        matrix = matrix.rename(index={pivotRow: pivotCol})
        pivotCol = matrix.loc['Z'].idxmin(axis=0, skipna=True)
        display(matrix)

#updating the table
#X lable updating 
print("The answer is: ")
X = ['X'+ str(i+1) for i in range(0,m)]
nash={}
#if we have X in the raw so .. this is our nash and we calcute it

for x in X:
    if x in matrix.index.values:
         nash[x] = round(matrix['b'].loc[x]/matrix['b'].loc['Z'], 4)
    else:
        nash[x] = 0
print(nash)


Enter the number of equations : 3
coefficients of objective function: 1 1 1
Enter the coefficient matrix with space: 
1 -1 3
3 5 -3
6 2 -2
Enter the resources: 1 1 1
The matrix with positive values is: 
[[ 5.  3.  7.]
 [ 7.  9.  1.]
 [10.  6.  2.]]


,X1,X2,X3,S1,S2,S3,b
S1,5.0,3.0,7.0,1.0,0.0,0.0,1.0
S2,7.0,9.0,1.0,0.0,1.0,0.0,1.0
S3,10.0,6.0,2.0,0.0,0.0,1.0,1.0
Z,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0


,X1,X2,X3,S1,S2,S3,b
S1,5,3,7,1,0,0,1
S2,7,9,1,0,1,0,1
S3,10,6,2,0,0,1,1
Z,-1,-1,-1,0,0,0,0


,X1,X2,X3,S1,S2,S3,b
S1,0.0,0.0,6.0,1.0,0.0,-0.5,0.5
S2,0.0,4.8,-0.4,0.0,1.0,-0.7,0.3
X1,1.0,0.6,0.2,0.0,0.0,0.1,0.1
Z,0.0,-0.4,-0.8,0.0,0.0,0.1,0.1


,X1,X2,X3,S1,S2,S3,b
S1,0,0,6,1,0,-0.5,0.5
S2,0,4.8,-0.4,0,1,-0.7,0.3
X1,1,0.6,0.2,0,0,0.1,0.1
Z,0,-0.4,-0.8,0,0,0.1,0.1


...

,X1,X2,X3,S1,S2,S3,b
X3,0,0,1,0.166667,0,-0.0833333,0.0833333
S2,0,4.8,0,0.0666667,1,-0.733333,0.333333
X1,1,0.6,0,-0.0333333,0,0.116667,0.0833333
Z,0,-0.4,0,0.133333,0,0.0333333,0.166667


,X1,X2,X3,S1,S2,S3,b
X3,0.0,0.0,1.0,0.166667,0.000000,-0.083333,0.083333
X2,0.0,1.0,0.0,0.013889,0.208333,-0.152778,0.069444
X1,1.0,0.0,0.0,-0.041667,-0.125000,0.208333,0.041667
Z,0.0,0.0,0.0,0.138889,0.083333,-0.027778,0.194444


,X1,X2,X3,S1,S2,S3,b
X3,0,0,1,0.166667,0,-0.0833333,0.0833333
X2,0,1,0,0.0138889,0.208333,-0.152778,0.0694444
X1,1,0,0,-0.0416667,-0.125,0.208333,0.0416667
Z,0,0,0,0.138889,0.0833333,-0.0277778,0.194444


,X1,X2,X3,S1,S2,S3,b
X3,0.400000,0.0,1.0,0.150000,-0.050000,0.0,0.1
X2,0.733333,1.0,0.0,-0.016667,0.116667,0.0,0.1
S3,4.800000,0.0,0.0,-0.200000,-0.600000,1.0,0.2
Z,0.133333,0.0,0.0,0.133333,0.066667,0.0,0.2


The answer is: 
{'X1': 0, 'X2': 0.5, 'X3': 0.5}
